In [1]:
import hpsklearn
import hyperopt.tpe
import hpsklearn
import hpsklearn.demo_support
import time
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from Project.dataStructures import *
from Project.manageDataset import *
from Project.dataStructures import *
from Project.dataStructures import get_data_target_lists
from Project.launchAlgorithms import *
from Project.utilities import *

WARN: OMP_NUM_THREADS=None =>
... If you are using openblas if you are using openblas set OMP_NUM_THREADS=1 or risk subprocess calls hanging indefinitely


/Library/Python/2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Library/Python/2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
rdm = np.random.RandomState()
rdm.seed(42)

In [3]:
datasetList, attributes, coloumns = read_file('/dataset.csv')
targetsDic, targets = get_classes(coloumns['class'])
dataset = string_to_float(switch_label(datasetList, targets))
rdm.shuffle(dataset)
labels = []
for item in dataset:
    labels.append(item[-1])  
stringLabels = [targets[item-1] for item in labels]

In [4]:
X = np.array(dataset)[:,:len(dataset[0])-1].astype(np.float32)
Y = np.array(dataset)[:,len(dataset[0])-1].astype(np.int32)
Y = np.array([item-1 for item in Y])
xTrain, xTest = X[:int(1000*0.8)], X[int(1000*0.8):]
yTrain, yTest = Y[:int(1000*0.8)], Y[int(1000*0.8):]


In [5]:
estimator = hpsklearn.HyperoptEstimator(
    preprocessing=hpsklearn.components.any_preprocessing('pp'),
    classifier=hpsklearn.components.any_classifier('clf'),
    algo=hyperopt.tpe.suggest,
    trial_timeout=15.0, # seconds
    max_evals=15,
    )

In [6]:
# Demo version of estimator.fit()
fit_iterator = estimator.fit_iter(xTrain,yTrain)
fit_iterator.next()
plot_helper = hpsklearn.demo_support.PlotHelper(estimator,
                                                mintodate_ylim=(-.01, .05))
while len(estimator.trials.trials) < estimator.max_evals:
    fit_iterator.send(1) # -- try one more model
    plot_helper.post_iter()
plot_helper.post_loop()

# -- Model selection was done on a subset of the training data.
# -- Now that we've picked a model, train on all training data.
estimator.retrain_best_model_on_full_data(xTrain, yTrain)

In [7]:
print 'Best preprocessing pipeline:'
for pp in estimator._best_preprocs:
    print pp
print
print 'Best classifier:\n', estimator.best_model()
test_predictions = estimator.predict(xTest)
acc_in_percent = 100 * np.mean(test_predictions == yTest)
print
print 'Prediction accuracy in generalization is %.1f%%' % acc_in_percent

Best preprocessing pipeline:

Best classifier:
{'learner': ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='sqrt', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=63, n_jobs=1, oob_score=False, random_state=1,
           verbose=False, warm_start=False), 'preprocs': (), 'ex_preprocs': ()}

Prediction accuracy in generalization is 63.3%
